In [ ]:
%load_ext autoreload
%autoreload 2

# Model design
import agentpy as ap
import networkx as nx
import random
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import IPython

from EnergyShedModel import EnergyShedModel

random.seed(1)

POP_SIZE = 3
NUM_NEIGHBORS = 6
NUM_STEPS = 100
GRID_SIZE = 2

In [ ]:
parameters = {"population": POP_SIZE, "number_of_neighbors": NUM_NEIGHBORS, "network_randomness": 0.5, "steps": NUM_STEPS, "grid_size": (GRID_SIZE, GRID_SIZE)}

model = EnergyShedModel(parameters)
results = model.run()

In [ ]:
results.reporters

In [ ]:
results.variables.EnergyShedModel

In [ ]:
color_map = {"labels": ["none", "buy", "sell"], "colors": ["b", "r", "g"]}

def EnergyShed_stackplot(data, ax):
    """Stackplot of people's condition over time."""
    x = data.index.get_level_values("t")
    y = [data[var] for var in ["none", "buy", "sell"]]

    sns.set()
    ax.stackplot(x, y, **color_map)

    ax.legend()
    ax.set_xlim(0, max(1, len(x) - 1))
    ax.set_ylim(0, 1)
    ax.set_xlabel("Time steps")
    ax.set_ylabel("Percentage of population")


fig, ax = plt.subplots()
EnergyShed_stackplot(results.variables.EnergyShedModel, ax)

In [ ]:
def animation_plot(m, axs):
    ax1, ax2 = axs
    ax1.set_title("EnergyShed spread")

    # Plot stackplot on first axis
    EnergyShed_stackplot(m.output.variables.EnergyShedModel, ax1)

    # Plot network on second axis
    color_dict = {0: "b", -1: "r", 1: "g"}
    colors = [color_dict[c] for c in m.agents.status]
    nx.draw_circular(m.network.graph, node_color=colors, node_size=50, ax=ax2)

fig, axs = plt.subplots(1, 2, figsize=(8, 4))  # Prepare figure
parameters["population"] = 50  # Lower population for better visibility
animation = ap.animate(EnergyShedModel(parameters), fig, axs, animation_plot)
IPython.display.HTML(animation.to_jshtml())

In [ ]:
def animation_plot(model, ax):
    group_grid = model.network.attr_grid('status')
    color_dict = {0: "b", -1: "r", 1: "g"}
    cmap = colors.ListedColormap([color_dict[key] for key in color_dict])  
    ap.gridplot(group_grid, cmap=cmap, ax=ax)
    ax.set_title(f"Energyshed model \n Time-step: {model.t}, "
                 f"Energy Transfer: {model.get_cost()},"
                 f"Weather: {model.get_weather()}")
    
fig, ax = plt.subplots()
model = EnergyShedModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())

# Experiments

In [ ]:
parameters = {
    "population": ap.IntRange(100, 1000),
    "number_of_neighbors": 4,
    "network_randomness": ap.Range(0.0, 1.0),
}

sample = ap.Sample(parameters, n=128, method="saltelli", calc_second_order=False)

In [ ]:
exp = ap.Experiment(EnergyShedModel, sample, iterations=10)
results = exp.run()

In [ ]:
results.save()

In [ ]:
results

In [ ]:
results.reporters.hist();

In [ ]:
# Sensitivity Analysis

In [ ]:
results.calc_sobol()

In [ ]:
def plot_sobol(results):
    """Bar plot of Sobol sensitivity indices."""

    sns.set()
    fig, axs = plt.subplots(1, 2, figsize=(8, 4))
    si_list = results.sensitivity.sobol.groupby(by="reporter")
    si_conf_list = results.sensitivity.sobol_conf.groupby(by="reporter")

    for (key, si), (_, err), ax in zip(si_list, si_conf_list, axs):
        si = si.droplevel("reporter")
        err = err.droplevel("reporter")
        si.plot.barh(xerr=err, title=key, ax=ax, capsize=3)
        ax.set_xlim(0)

    axs[0].get_legend().remove()
    axs[1].set(ylabel=None, yticklabels=[])
    axs[1].tick_params(left=False)
    plt.tight_layout()


plot_sobol(results)

In [ ]:
def plot_sensitivity(results):
    """Show average simulation results for different parameter values."""

    sns.set()
    fig, axs = plt.subplots(2, 2, figsize=(8, 8))
    axs = [i for j in axs for i in j]  # Flatten list

    data = results.arrange_reporters().astype("float")
    params = results.parameters.sample.keys()

    for x, ax in zip(params, axs):
        for y in results.reporters.columns:
            sns.regplot(x=x, y=y, data=data, ax=ax, ci=99, x_bins=15, fit_reg=False, label=y)
        ax.set_ylim(0, 1)
        ax.set_ylabel("")
        ax.legend()

    plt.tight_layout()


plot_sensitivity(results)